# HW 4 Graph Coloring in SAT

We will start with simple encoding of CNF formulae to illustrate how CNFs are encoded for typical SAT solvers. The classical encoding is to make a list of lists of numbers with each unique integer representing a Boolean literal. If the number is negative, the literal is negated. Each element of the list is a clause and each element of a clause is a literal.

In [ ]:
# get the sat solver...
import pycosat

For example: 
1. $(x) \land (x \lor y)$
2. $(x \lor y) \land (x \lor y')$
3. $(x) \land (y) \land (x' \lor y' \lor z) \land (x' \lor z')$

In [ ]:
cnf1 = [[1],[1,2]]
cnf2 = [[1,2],[1,-2]]
cnf3 = [[1],[2],[-1,-2,3],[-1,-3]]

We can pass a clause to PycoSAT directly...

In [ ]:
print(pycosat.solve(cnf1))
print(pycosat.solve(cnf2))
print(pycosat.solve(cnf3))

List comprehension makes short work of regular patterns of clauses:
 
6. $x_1 \lor x_2 \lor \dots \lor x_n$
7. $x'_1 \land x'_2 \land \dots \land x'_n$
8. $(x_1 \lor x'_2) \land (x_1 \lor x'_3) \land \dots \land (x_1 \lor x'_n) \land (x_2 \lor x'_3) \land \dots \land (x_i \lor x'_j) \land \dots \land (x_{n-1} \lor x'_n)$

In [ ]:
n = 10
cnf6 = [[i for i in range(1,n)]]
cnf7 = [[-i] for i in range(1,n)]
cnf8 = [[i,-j] for i in range(1,n-1) for j in range(i+1,n)]

print(cnf6)
print(pycosat.solve(cnf6))
print(cnf7)
print(pycosat.solve(cnf7))
print(cnf8)
print(pycosat.solve(cnf8))

### Random Graph R(n,~k)
To test your coloring algorithm, you need to create random simple graphs (no edges from a node to itself) with N nodes and K undirected edges. There are $N (N-1)/2 $ possible edges in a simple graph. Building a genuinely random graph is tricky, so instead, we will make a graph with N verticies and usually K edges by choosing each possible edge with probability of $2K/(N(N-1))$. Your function should set N = # of nodes, M = #actual number of edges, and edge = list of edge lists. For example:

In [ ]:
N = 10
M = 30
edge = [[0,1],[0,2],[0,3],[0,5],[0,6],[0,7],[1,2],[1,4],[1,5],[1,8],[1,9],
        [2,4],[2,5],[2,7],[2,8],[2,9],[3,4],[3,5],[3,7],[3,8],[4,6],[4,7],
        [4,9],[5,6],[5,7],[6,7],[6,8],[6,9],[7,8],[7,9]]

In [ ]:
# Import Pandas, numpy and Networkx
import numpy as np
import pandas as pd
import networkx as nx

# SOME PROBLEM WITH SCIPY AND DEPENDECIES
# import scipy as sp

In [ ]:
import random
import matplotlib.pyplot as plt
from matplotlib import interactive
interactive(True)

In [ ]:
G = nx.Graph()
G.add_edges_from(edge)
G.add_nodes_from([i for i in range(0,N)])

In [ ]:
nx.draw(G, with_labels=True, font_weight='bold')
plt.show()
A = nx.adjacency_matrix(G)
Adj = A.todense()
Adj

### Problem 1
Build a SAT based coloring algorithm. A coloring is a setting of each node to a color such that for the K colors used, no edge connects a node of a given color to another of the same color. It is probably a good idea to build the constraints separately by defining an empty list of clauses, and then appending new clauses for each constraint.
One simple approach: $x_{i,j} = 1$ iff vertex $i$ is color $j$ then
* Each vertex gets a color
* Nodes on each edge get different colors
* Only one color per vertex
Find the minimal number of colors for the example above. Note, for larger examples, you might need to think about how to want to search for the minimum. (You may find that a satisfiable solution is much quicker than an unsatisfiable one).

In [ ]:
# IDEA is to iteratively search for minimum number of colors that can still 
# satisfy the constraints

# Best solution is binary search, keep going left until you reach unsatsifable and left=right

num_colors = N

while num_colors >=2 :

    color_list = [i for i in range(num_colors)]

    # Defining a list(length = num_Colors) of binary variables for each node -- AS SAT problem
    # Zero-based indexing doesn't work --> -0 =0
    # One-based indexing

    cnf1 = []

    # Constraint 1: Every vertex can only get a unique color
    for i in range(0,N): # Every node
        for j in range(1,num_colors): # Traverse through color options --> equivalent number of constraints for a single node
            for k in range(j+1,num_colors+1):
                    cnf1.append([-(i*num_colors + j), -(i*num_colors + k) ])


    # Constraint 2: Choose one of the color in the palette
    for i in range(0,N):         
            cnf1.append([i*num_colors + j for j in range(1, num_colors+1)])

    # Constraint 3: For all edges: no two vertex share same colour
    for e in edge:
        # e[0] -> gives left-vertex, e[1] -> gives right-vertex
        for j in range(1,num_colors+1):
            cnf1.append([-((e[0])*num_colors +j), -((e[1])*num_colors +j)])
    
    print(cnf1)
    tmp = pycosat.solve(cnf1)
    if tmp == "UNSAT":
        break
    solution = tmp
    num_colors -= 1
    
num_colors += 1
print (solution)
print("Minimum num_colors is:", num_colors)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
import itertools
colors = dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)
colors = dict(itertools.islice(colors.items(), num_colors))
color_vals = list(colors.values())
print(color_vals)

In [ ]:
vertex_color = []
for node in range(N):
    for j in range(0, num_colors):
        index = node*num_colors + j
        if solution[index] > 0:
            print("index here is:", index)
            vertex_color.append(color_vals[(solution[index] -1) % (num_colors)])
print(vertex_color)

In [ ]:
# The networkx API has some weird bugs: HAD to add nodelist additionally in arguments
nx.draw(G, with_labels=True, font_weight='bold', nodelist=[i for i in range(0,N)], node_color = vertex_color )
plt.show() 

### Problem 2
Using your random graph generator, investigate the chromatic number (i.e. the minimum number of colors to color a graph) for graphs of ascending N with edge density $d$ = [1/8, 1/4, 1/3, 1/2, 2/3]. Run sufficient cases to estimate a mean, upper and lower bound on the chromatic number for N and d. Keep track of your run-times for the cases. Do you see a trend in run-time complexity?

In [ ]:
edge.clear()
N =10
k = 30

def randgraph(N, d):
    
    for i in range(N):
        for j in range(N):
            if i!=j and random.randint(0, 1) <= d :
                edge.append([i,j])
            
    return N, len(edge), edge            

In [ ]:
import time
from time import process_time
from numpy import mean
import numpy as np

In [ ]:
density_list = [1/8, 1/4, 1/3, 1/2, 2/3]
Nstart = 5
Nend = 15
Nstep = 5
runs = 5
exec_times_N_d = []
chr_num_N_min = []
chr_num_N_max = []
chr_num_N_avg = []
exec_time_per_N = []
for N in range (Nstart, Nend+1, Nstep): 
    
    exec_time_for_d = []
    chr_num_per_d_min = []
    chr_num_per_d_max = []
    chr_num_per_d_avg = []
    
    for d in density_list:
        Chr_num_per_run= []        
        start = time.time()        
        for k in range(runs):
            edge.clear()
            
            print(" ############################# N =", N, "d =", d, "Run =", k+1, "###############################")
            N, M, edge = randgraph(N,d)
            G = nx.Graph()
            G.add_edges_from(edge)
            G.add_nodes_from([i for i in range(0,N)])
            nx.draw(G, with_labels=True, font_weight='bold')
            plt.title('Before Coloring')
            plt.show() 

                # IDEA is to iteratively search for minimum number of colors that can still 
            # satisfy the constraints

            # Best solution is binary search, keep going left until you reach unsatsifable and left=right

            num_colors = N

            while num_colors >=2 :

                color_list = [i for i in range(num_colors)]

                # Defining a list(length = num_Colors) of binary variables for each node -- AS SAT problem
                # Zero-based indexing doesn't work --> -0 =0
                # One-based indexing

                cnf1 = []

                # Constraint 1: Every vertex can only get a unique color
                for i in range(0,N): # Every node
                    for j in range(1,num_colors): # Traverse through color options --> equivalent number of constraints for a single node
                        for k in range(j+1,num_colors+1):
                                cnf1.append([-(i*num_colors + j), -(i*num_colors + k) ])


                # Constraint 2: Choose one of the color in the palette
                for i in range(0,N):         
                        cnf1.append([i*num_colors + j for j in range(1, num_colors+1)])

                # Constraint 3: For all edges: no two vertex share same colour
                for e in edge:
                    # e[0] -> gives left-vertex, e[1] -> gives right-vertex
                    for j in range(1,num_colors+1):
                        cnf1.append([-((e[0])*num_colors +j), -((e[1])*num_colors +j)])

                # print(cnf1)
                tmp = pycosat.solve(cnf1)
                if tmp == "UNSAT":
                    break
                solution = tmp
                num_colors -= 1

            num_colors += 1
            # print (solution)
            print("N = ", N, "Min_colors", num_colors)


            colors = dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)
            colors = dict(itertools.islice(colors.items(), num_colors))
            color_vals = list(colors.values())
            print(color_vals)

            vertex_color = []
            for node in range(N):
                for j in range(0, num_colors):
                    index = node*num_colors + j
                    if solution[index] > 0:
                        # print("index here is:", index)
                        vertex_color.append(color_vals[(solution[index] -1) % (num_colors)])
            # print(vertex_color)

            # The networkx API has some weird bugs: HAD to add nodelist additionally in arguments
            nx.draw(G, with_labels=True, font_weight='bold', nodelist=[i for i in range(0,N)], node_color = vertex_color )
            plt.title('After coloring')
            plt.show()
        
            Chr_num_per_run.append(num_colors)
            
        end_time = time.time()
        exec_time = end_time - start
        exec_time_for_d.append(exec_time/ runs)
        
        chr_num_per_d_min.append(min(Chr_num_per_run))
        chr_num_per_d_max.append(max(Chr_num_per_run))
        chr_num_per_d_avg.append(np.mean(Chr_num_per_run))
        
    chr_num_N_min.append(chr_num_per_d_min)
    chr_num_N_max.append(chr_num_per_d_max)
    chr_num_N_avg.append(chr_num_per_d_avg)  
    exec_times_N_d.append(exec_time_for_d)
      

In [ ]:
exec_times_N_d

In [ ]:
X = [density_list for i in range(Nstart, Nend +1, Nstep)]
X

In [ ]:
try:
    import matplotlib.pyplot as plt
    from pylab import rcParams
    %matplotlib inline
    rcParams['figure.figsize'] = 10, 6
    
    # First plot : Execution times
    
    print ( "############### EXECUTION TIME PLOTS #################" )
    plt.figure()
    for i in range(len(X)):
        plt.plot(X[i], exec_times_N_d[i], label='N= %d' %(Nstart +Nstep*i) )
    plt.xlabel('Edge densities')
    plt.ylabel('Average execution times in seconds')
    plt.legend([i for i in range(Nstart, Nend +1, Nstep)])
    plt.show()
    
        # First plot : Minimum chromatic number
    
    print ( "############### Min chromatic number PLOTS #################" )
    plt.figure()
    for i in range(len(X)):
        plt.plot(X[i], chr_num_N_min[i], label='N= %d' %(Nstart +Nstep*i) )
    plt.xlabel('Edge densities')
    plt.ylabel('Minimum chromatic number')
    plt.show()
    
            # First plot : Maximum chromatic number
    
    print ( "############### Max chromatic number PLOTS #################" )
    plt.figure()
    for i in range(len(X)):
        plt.plot(X[i], chr_num_N_max[i], label='N= %d' %(Nstart +Nstep*i) )
    plt.xlabel('Edge densities')
    plt.ylabel('Max chromatic number')
    plt.show()
    
            # First plot : Average chromatic number
    
    print ( "############### Avg chromatic number PLOTS #################" )
    plt.figure()
    for i in range(len(X)):
        plt.plot(X[i], chr_num_N_avg[i], label='N= %d' %(Nstart +Nstep*i) )
    plt.xlabel('Edge densities')
    plt.ylabel('Average chromatic number')
    plt.show()
    

except ImportError:
    print("matplotlib not found, nothing will be displayed")
    plt = None
    def display_solution(sol):   pass

### Problem 3
One issue in generation of chromatic numbers i is that the color values are symmetric in the sense that a minimal solution of 6 colors has 6! permutations of equivalent colorings. This leads to a problem for unsatisfiable trials where all k! permutations are tested before admitting failure.
A solution to this problem is symmetry breaking -- adding additional constraints that limit permutations of the solution space. Using your results from Problem 2, create 5 problems that require more than 1 second to color. The simplest form of symmetry breaking is to fix vertex 0 with color 0. What effect, if any, does that have on your 5 test cases?

Can you find a more general symmetry breaking technique that reduces the run-time but does not harm the results? Describe your technique clearly and run sufficient examples to verify its correctness.

In [ ]:
density_list = [1/2, 2/3]
Nstart = 20
Nend = 20
Nstep = 5
runs = 5
exec_times_N_d = []
chr_num_N_min = []
chr_num_N_max = []
chr_num_N_avg = []
exec_time_per_N = [] 

activate_constraint = 1

for N in range (Nstart, Nend+1, Nstep): 
    
    exec_time_for_d = []
    chr_num_per_d_min = []
    chr_num_per_d_max = []
    chr_num_per_d_avg = []
    
    for d in density_list:
        Chr_num_per_run= []        
        start = time.time()        
        for k in range(runs):
            edge.clear()
            
            print(" ############################# N =", N, "d =", d, "Run =", k+1, "###############################")
            N, M, edge = randgraph(N,d)
            G = nx.Graph()
            G.add_edges_from(edge)
            G.add_nodes_from([i for i in range(0,N)])
            nx.draw(G, with_labels=True, font_weight='bold')
            plt.title('Before Coloring')
            plt.show() 

                # IDEA is to iteratively search for minimum number of colors that can still 
            # satisfy the constraints

            # Best solution is binary search, keep going left until you reach unsatsifable and left=right

            num_colors = N

            while num_colors >=2 :

                color_list = [i for i in range(num_colors)]
                
                # Defining a list(length = num_Colors) of binary variables for each node -- AS SAT problem
                # Zero-based indexing doesn't work --> -0 =0
                # One-based indexing

                cnf1 = []
                
                if activate_constraint == 1:
                # CONSTRAINT NEW : FIX COLOR FOR NODE 0                
                    cnf1.append([1])
                
                # Constraint 1: Every vertex can only get a unique color
                for i in range(0,N): # Every node
                    for j in range(1,num_colors): # Traverse through color options --> equivalent number of constraints for a single node
                        for k in range(j+1,num_colors+1):
                                cnf1.append([-(i*num_colors + j), -(i*num_colors + k) ])


                # Constraint 2: Choose one of the color in the palette
                for i in range(0,N):         
                        cnf1.append([i*num_colors + j for j in range(1, num_colors+1)])

                # Constraint 3: For all edges: no two vertex share same colour
                for e in edge:
                    # e[0] -> gives left-vertex, e[1] -> gives right-vertex
                    for j in range(1,num_colors+1):
                        cnf1.append([-((e[0])*num_colors +j), -((e[1])*num_colors +j)])

                # print(cnf1)
                tmp = pycosat.solve(cnf1)
                if tmp == "UNSAT":
                    break
                solution = tmp
                num_colors -= 1

            num_colors += 1
            # print (solution)
            print("N = ", N, "Min_colors", num_colors)


            colors = dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)
            colors = dict(itertools.islice(colors.items(), num_colors))
            color_vals = list(colors.values())
            print(color_vals)

            vertex_color = []
            for node in range(N):
                for j in range(0, num_colors):
                    index = node*num_colors + j
                    if solution[index] > 0:
                        # print("index here is:", index)
                        vertex_color.append(color_vals[(solution[index] -1) % (num_colors)])
            # print(vertex_color)

            # The networkx API has some weird bugs: HAD to add nodelist additionally in arguments
            nx.draw(G, with_labels=True, font_weight='bold', nodelist=[i for i in range(0,N)], node_color = vertex_color )
            plt.title('After coloring')
            plt.show()
        
            Chr_num_per_run.append(num_colors)
            
        end_time = time.time()
        exec_time = end_time - start
        exec_time_for_d.append(exec_time/ runs)
        
        chr_num_per_d_min.append(min(Chr_num_per_run))
        chr_num_per_d_max.append(max(Chr_num_per_run))
        chr_num_per_d_avg.append(np.mean(Chr_num_per_run))
        
    chr_num_N_min.append(chr_num_per_d_min)
    chr_num_N_max.append(chr_num_per_d_max)
    chr_num_N_avg.append(chr_num_per_d_avg)  
    exec_times_N_d.append(exec_time_for_d)
      

In [ ]:
exec_times_N_d

In [ ]:
# DIFFERENCE NOTED: BY TRIVIAL CONSTRAINT ADDITION
# ALMOST 1/3 BY FIXING 1-COLOR

before_iter_constraint_exec_times = [[60.668498039245605, 1.643794870376587]]
after_iter_constraint_exec_times = [[18.411622285842896, 0.5934930801391601]]


In [ ]:
'''
# NEW IDEA: FRUGAL ASSIGNMENT (FIXING) OF COLORS ITERATIVELY from node 0 to N-1, with condition
# CONDITION: cannot assign color of already considered node that also shares edge with current node
# FOR EVERY EDGE, CHECK IF THE PALETTE IS EXHAUSTED
# FIND THE MINIMUM REQUIRED COLOR PALETTE: DYNAMIC PROGRAMMING ..?

density_list = [1/2]
Nstart = 10
Nend = 10
Nstep = 5
runs = 1
exec_times_N_d = []
chr_num_N_min = []
chr_num_N_max = []
chr_num_N_avg = []
exec_time_per_N = [] 

for N in range (Nstart, Nend+1, Nstep): 
    
    exec_time_for_d = []
    chr_num_per_d_min = []
    chr_num_per_d_max = []
    chr_num_per_d_avg = []
    
    for d in density_list:
        Chr_num_per_run= []        
        start = time.time()        
        for k in range(runs):
            edge.clear()
            
            print(" ############################# N =", N, "d =", d, "Run =", k+1, "###############################")
            N, M, edge = randgraph(N,d)
            G = nx.Graph()
            G.add_edges_from(edge)
            G.add_nodes_from([i for i in range(0,N)])
            nx.draw(G, with_labels=True, font_weight='bold')
            plt.title('Before Coloring')
            plt.show() 

            # IDEA is to iteratively search for minimum number of colors that can still satisfy the constraints

            for available_colors in range(1,N+1):
                
                cnf1 = []
                color_list = [i for i in range(available_colors)] # CURRENTLY AVAIABLE PALETTE

                # CONSTRAINT NEW : FIX COLOR FOR NODE 0                
                cnf1.append([1])    
                for j in range(1,available_colors): # Traverse through color options --> equivalent number of constraints for a single node
                        for k in range(j+1,available_colors+1):
                                cnf1.append([-j, -k ])
                
                for i in range(0,N): # Every node
                
                # MAINTAIN A LIST OF PALETTE for each node, and remove the colors that cannot be assigned
                # CHOOSE THE Oth indexed after deletions, KEEP AN EYE FOR NOT SATISFIABLE
                color_palette = [ j for j in range(1,N+1) ]
                
                # Additional Constraint 1: Every vertex gets a unique color COMPARED TO ITS EDGES
                
                 # GET THE ADJACENCY LIST for EACH NODE
                
                    for j in range(1,num_colors): # Traverse through color options --> equivalent number of constraints for a single node
                        for k in range(j+1,num_colors+1):
                                cnf1.append([-(i*num_colors + j), -(i*num_colors + k) ])


                # Constraint 2: Choose one of the color in the palette
                for i in range(0,N):         
                        cnf1.append([i*num_colors + j for j in range(1, num_colors+1)])

                # Constraint 3: For all edges: no two vertex share same colour
                for e in edge:
                    # e[0] -> gives left-vertex, e[1] -> gives right-vertex
                    for j in range(1,num_colors+1):
                        cnf1.append([-((e[0])*num_colors +j), -((e[1])*num_colors +j)])

                # print(cnf1)
                tmp = pycosat.solve(cnf1)
                if tmp == "UNSAT":
                    break
                solution = tmp
                num_colors -= 1

            num_colors += 1
            # print (solution)
            print("N = ", N, "Min_colors", num_colors)


            colors = dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)
            colors = dict(itertools.islice(colors.items(), num_colors))
            color_vals = list(colors.values())
            print(color_vals)

            vertex_color = []
            for node in range(N):
                for j in range(0, num_colors):
                    index = node*num_colors + j
                    if solution[index] > 0:
                        # print("index here is:", index)
                        vertex_color.append(color_vals[(solution[index] -1) % (num_colors)])
            # print(vertex_color)

            # The networkx API has some weird bugs: HAD to add nodelist additionally in arguments
            nx.draw(G, with_labels=True, font_weight='bold', nodelist=[i for i in range(0,N)], node_color = vertex_color )
            plt.title('After coloring')
            plt.show()
        
            Chr_num_per_run.append(num_colors)
            
        end_time = time.time()
        exec_time = end_time - start
        exec_time_for_d.append(exec_time/ runs)
        
        chr_num_per_d_min.append(min(Chr_num_per_run))
        chr_num_per_d_max.append(max(Chr_num_per_run))
        chr_num_per_d_avg.append(np.mean(Chr_num_per_run))
        
    chr_num_N_min.append(chr_num_per_d_min)
    chr_num_N_max.append(chr_num_per_d_max)
    chr_num_N_avg.append(chr_num_per_d_avg)  
    exec_times_N_d.append(exec_time_for_d)
    
'''
      

In [ ]:
density_list = [1/3, 1/2, 2/3]
Nstart = 20
Nend = 28
Nstep = 5
runs = 5
exec_times_N_d = []
chr_num_N_min = []
chr_num_N_max = []
chr_num_N_avg = []
exec_time_per_N = [] 

activate_constraint = 1

for N in range (Nstart, Nend+1, Nstep): 
    
    exec_time_for_d = []
    chr_num_per_d_min = []
    chr_num_per_d_max = []
    chr_num_per_d_avg = []
    
    for d in density_list:
        Chr_num_per_run= []        
        start = time.time()        
        for k in range(runs):
            edge.clear()
            
            print(" ############################# N =", N, "d =", d, "Run =", k+1, "###############################")
            N, M, edge = randgraph(N,d)
            G = nx.Graph()
            G.add_edges_from(edge)
            G.add_nodes_from([i for i in range(0,N)])
            A = nx.adjacency_matrix(G)
            Adj = nx.to_numpy_array(G)
            nx.draw(G, with_labels=True, font_weight='bold')
            plt.title('Before Coloring')
            plt.show() 
            # print(Adj)
            
 #           res = [sum(i) for i in zip(*Adj) ]
 #           max_value = max(res)
 #           max_index = res.index(max_value)
 #          print(max_index)
            
            max_sum= 0
            clique_node = 1
            for i in range(len(Adj)) :
                Sum = sum(Adj[i])
            #     print(Sum)
                if Sum > max_sum:
                    max_sum = Sum
                    clique_node = i+1

            print("Clique Node is", clique_node)
           # print("Clique connections are", Adj[clique_node - 1])
            
            index = 0
            next_clique_node = 1
            max_sum = 0
            for connection in Adj[clique_node - 1]:
                
                if connection ==1:                    
                    candidate_node = index
                    Sum = sum(Adj[index])

                    print(Sum)
                    if Sum > max_sum:
                        max_sum = Sum
                        next_clique_node = index + 1
               #         print("Index here is", index)
                index += 1  
                
            print("Next clique node is", next_clique_node)
            
                # IDEA is to iteratively search for minimum number of colors that can still 
            # satisfy the constraints

            # Best solution is binary search, keep going left until you reach unsatsifable and left=right

            num_colors = N

            while num_colors >=2 :

                color_list = [i for i in range(num_colors)]
                
                # Defining a list(length = num_Colors) of binary variables for each node -- AS SAT problem
                # Zero-based indexing doesn't work --> -0 =0
                # One-based indexing

                cnf1 = []
                
                if activate_constraint == 1:
                # CONSTRAINT NEW : FIX COLOR FOR NODE 0                
                    cnf1.append([(clique_node-1)*num_colors + 1])
                    cnf1.append([(next_clique_node-1)*num_colors + 2])
                # Constraint 1: Every vertex can only get a unique color
                for i in range(0,N): # Every node
                    for j in range(1,num_colors): # Traverse through color options --> equivalent number of constraints for a single node
                        for k in range(j+1,num_colors+1):
                                cnf1.append([-(i*num_colors + j), -(i*num_colors + k) ])


                # Constraint 2: Choose one of the color in the palette
                for i in range(0,N):         
                        cnf1.append([i*num_colors + j for j in range(1, num_colors+1)])

                # Constraint 3: For all edges: no two vertex share same colour
                for e in edge:
                    # e[0] -> gives left-vertex, e[1] -> gives right-vertex
                    for j in range(1,num_colors+1):
                        cnf1.append([-((e[0])*num_colors +j), -((e[1])*num_colors +j)])

                # print(cnf1)
                tmp = pycosat.solve(cnf1)
                if tmp == "UNSAT":
                    break
                solution = tmp
                num_colors -= 1

            num_colors += 1
            # print (solution)
            print("N = ", N, "Min_colors", num_colors)


            colors = dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)
            colors = dict(itertools.islice(colors.items(), num_colors))
            color_vals = list(colors.values())
            print(color_vals)

            vertex_color = []
            for node in range(N):
                for j in range(0, num_colors):
                    index = node*num_colors + j
                    if solution[index] > 0:
                        # print("index here is:", index)
                        vertex_color.append(color_vals[(solution[index] -1) % (num_colors)])
            # print(vertex_color)

            # The networkx API has some weird bugs: HAD to add nodelist additionally in arguments
            nx.draw(G, with_labels=True, font_weight='bold', nodelist=[i for i in range(0,N)], node_color = vertex_color )
            plt.title('After coloring')
            plt.show()
        
            Chr_num_per_run.append(num_colors)
            
        end_time = time.time()
        exec_time = end_time - start
        exec_time_for_d.append(exec_time/ runs)
        
        chr_num_per_d_min.append(min(Chr_num_per_run))
        chr_num_per_d_max.append(max(Chr_num_per_run))
        chr_num_per_d_avg.append(np.mean(Chr_num_per_run))
        
    chr_num_N_min.append(chr_num_per_d_min)
    chr_num_N_max.append(chr_num_per_d_max)
    chr_num_N_avg.append(chr_num_per_d_avg)  
    exec_times_N_d.append(exec_time_for_d)
      

In [ ]:
X = [density_list for i in range(Nstart, Nend +1, Nstep)]
try:
    import matplotlib.pyplot as plt
    from pylab import rcParams
    %matplotlib inline
    rcParams['figure.figsize'] = 10, 6
    
    # First plot : Execution times
    
    print ( "############### EXECUTION TIME PLOTS #################" )
    plt.figure()
    for i in range(len(X)):
        plt.plot(X[i], exec_times_N_d[i], label='N= %d' %(Nstart +Nstep*i) )
    plt.xlabel('Edge densities')
    plt.ylabel('Average execution times in seconds')
    plt.legend([i for i in range(Nstart, Nend +1, Nstep)])
    plt.show()
    
        # First plot : Minimum chromatic number
    
    print ( "############### Min chromatic number PLOTS #################" )
    plt.figure()
    for i in range(len(X)):
        plt.plot(X[i], chr_num_N_min[i], label='N= %d' %(Nstart +Nstep*i) )
    plt.xlabel('Edge densities')
    plt.ylabel('Minimum chromatic number')
    plt.show()
    
            # First plot : Maximum chromatic number
    
    print ( "############### Min chromatic number PLOTS #################" )
    plt.figure()
    for i in range(len(X)):
        plt.plot(X[i], chr_num_N_max[i], label='N= %d' %(Nstart +Nstep*i) )
    plt.xlabel('Edge densities')
    plt.ylabel('Minimum chromatic number')
    plt.show()
    
            # First plot : Average chromatic number
    
    print ( "############### Min chromatic number PLOTS #################" )
    plt.figure()
    for i in range(len(X)):
        plt.plot(X[i], chr_num_N_avg[i], label='N= %d' %(Nstart +Nstep*i) )
    plt.xlabel('Edge densities')
    plt.ylabel('Minimum chromatic number')
    plt.show()
    

except ImportError:
    print("matplotlib not found, nothing will be displayed")
    plt = None
    def display_solution(sol):   pass